# 📖 [LMX Tutorial] Absolute and Relative motion (in C++)

Based on the following WMX API document:

http://download.movensys.com:8222/doc/wmx3/en/html/api/page_WMXDOC_TUTORIAL_SEC3_1_ABS_REL.html

### 📢Important note about xeus-cling
A Jupyter kernel for C++ based on the `cling` C++ interpreter and the `xeus` native implementation of the Jupyter protocol, xeus.

To understand how the xeus-clang works, please try to run the following sample notebook provided by xeus-cling in advance.

Link to [xeus-cling notebook sample](xeus-cling-notebooks/xcpp.ipynb)

## Mandatory: include a LMX cling header file to load WMX libraries

In [1]:
// Load necessary libraries that are required by WMX3 API first
#include <lmx/lmx_cling_config.hpp>

## Import WMX library and Initialize a WMX3 API device

In [2]:
#include <iostream>
#include <stdexcept>

// When all the devices are done, the WMX3 engine will also terminate.
#include "WMX3Api.h"
#include "CoreMotionApi.h"

using namespace wmx3Api;

WMX3Api wmx3Lib;
CoreMotionStatus cmStatus;
CoreMotion wmx3LibCore(&wmx3Lib);

// Declare variables to handle a return value
int     ret;
char    errString[256];

std::cout << "Program Start" << std::endl;

// Create device.
ret = wmx3Lib.CreateDevice("/opt/lmx/", DeviceType::DeviceTypeNormal, INFINITE);
if (ret != ErrorCode::None) {
    std::cerr << "Failed to execute CreateDevice. Error=" << ret << std::endl;
    throw std::runtime_error("WMX API exception");
}

std::cout << "CreateDevice succeeded!" << std::endl;

Program Start
CreateDevice succeeded!


## Set the device name and start communication

In [3]:
// Set Device Name.
wmx3Lib.SetDeviceName("MotorControl");
if (ret != ErrorCode::None) {
    std::cerr << "Failed to execute SetDeviceName. Error=" << ret << std::endl;
    throw std::runtime_error("WMX API exception");
}

// Start Communication.
wmx3Lib.StartCommunication(INFINITE);
if (ret != ErrorCode::None) {
    std::cerr << "Failed to execute StartCommunication. Error=" << ret << std::endl;
    throw std::runtime_error("WMX API exception");
}

std::cout << "StartCommunication succeeded!" << std::endl;

StartCommunication succeeded!


In [4]:
// Get DevicesInfo to determine the type of device currently created
wmx3Api::DevicesInfoA devInfo;

// Set wmx3Lib Name.
ret = wmx3Lib.GetAllDevices(&devInfo);
if (ret != ErrorCode::None) {
    std::cerr << "Failed to execute GetAllDevices. Error=" << ret << std::endl;
    throw std::runtime_error("WMX API exception");
}

std::cout << "device Id : " << devInfo.devices[0].id << " Name : " << devInfo.devices[0].name << std::endl;


device Id : 0 Name : MotorControl


## Set servo ON ▶️

In [5]:
// Set servo on.
wmx3LibCore.axisControl->SetServoOn(0, 1);
while (true)
{
    wmx3LibCore.GetStatus(&cmStatus);
    if (cmStatus.axesStatus[0].servoOn)
    {
        break;
    }

    Sleep(100);
}

## 1. Absolute position command (StartPos)

Refer to the following documentation:

http://download.movensys.com:8222/doc/wmx3/en/html/api/classwmx3_api_1_1_motion.html#_CPPv4N7wmx3Api6Motion8StartPosEjP17TriggerPosCommand

In [7]:
Motion::PosCommand pos;

//Set position command parameters
pos.axis = 0;
pos.profile.type = ProfileType::Trapezoidal;
pos.profile.velocity = 10000;
pos.profile.acc = 10000;
pos.profile.dec = 10000;

//Execute absolute position command to 10000
pos.target = 10000;
ret = wmx3LibCore.motion->StartPos(&pos);
if (ret != ErrorCode::None) {
    wmx3LibCore.ErrorToString(ret, errString, sizeof(errString));
    std::cerr << "Failed to execute StartPos. Error=" << ret << " (" << errString << ")" << std::endl;
    throw std::runtime_error("WMX API exception");
}

//Block execution until motion is finished
ret = wmx3LibCore.motion->Wait(0);
if (ret != ErrorCode::None) {
    wmx3LibCore.ErrorToString(ret, errString, sizeof(errString));
    std::cerr << "Failed to execute Wait. Error=" << ret << " (" << errString << ")" << std::endl;
    throw std::runtime_error("WMX API exception");
}

std::cout << "StartPos succeeded!" << std::endl;

StartPos succeeded!


### 1.1 Draw the position and velocity plots for relative motion 📈

In [ ]:
// DrawLogPlot("Relative position command (StartMov)", False);